In [1]:
import torch
import torch.nn as nn
from cmapPy.pandasGEXpress.parse import parse
import data_loader as dl
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import pandas as pd


import data_loader as dl
import autoencoder
import glob

In [28]:
#param_path = "trained_model_params/modZ_input/0.00237588_leakyReLu_960_1000_model_best.pth"
files

['trained_model_params/modZ_input\\0.00233913_leakyReLu_960_1400_model_best.pth',
 'trained_model_params/modZ_input\\0.00236055_leakyReLu_960_1200_model_best.pth',
 'trained_model_params/modZ_input\\0.00236989_leakyReLu_960_1300_model_best.pth',
 'trained_model_params/modZ_input\\0.00237588_leakyReLu_960_1000_model_best.pth',
 'trained_model_params/modZ_input\\0.00238399_leakyReLu_960_1100_model_best.pth',
 'trained_model_params/modZ_input\\0.09155359_leakyReLu_960_700_model_best.pth']

In [3]:
data_path = "input_data/HF_LM_genes_DEG_data.csv"
pEF_vector_in = np.array(pd.read_csv(data_path).z_score_PEF)
pEF_vector_in = torch.from_numpy(pEF_vector_in).double()

files = glob.glob("trained_model_params/modZ_input/*.pth")
criterion = nn.MSELoss()

for f in files:
    
    # Load parameters from training
    params = torch.load(f)
    
    # Get the dimensions of the first layer and the weights
    latent_size, input_size = params['state_dict']['net.0.weight'].shape
    
    AE = autoencoder.Autoencoder(input_size,latent_size)
    AE.set_weights(params)
    AE.double()
    AE.cpu()

    pEF_vector_out = AE(pEF_vector_in)

    loss = criterion(pEF_vector_in, pEF_vector_out)
    print(f)
    print(loss)
    


trained_model_params/modZ_input\0.00233913_leakyReLu_960_1400_model_best.pth
tensor(0.0002, dtype=torch.float64, grad_fn=<MseLossBackward0>)
trained_model_params/modZ_input\0.00236055_leakyReLu_960_1200_model_best.pth
tensor(0.0004, dtype=torch.float64, grad_fn=<MseLossBackward0>)
trained_model_params/modZ_input\0.00236989_leakyReLu_960_1300_model_best.pth
tensor(0.0004, dtype=torch.float64, grad_fn=<MseLossBackward0>)
trained_model_params/modZ_input\0.00237588_leakyReLu_960_1000_model_best.pth
tensor(0.0003, dtype=torch.float64, grad_fn=<MseLossBackward0>)
trained_model_params/modZ_input\0.00238399_leakyReLu_960_1100_model_best - Copy.pth
tensor(0.0002, dtype=torch.float64, grad_fn=<MseLossBackward0>)
trained_model_params/modZ_input\0.00238399_leakyReLu_960_1100_model_best.pth
tensor(0.0002, dtype=torch.float64, grad_fn=<MseLossBackward0>)
trained_model_params/modZ_input\0.09155359_leakyReLu_960_700_model_best.pth
tensor(6.0070, dtype=torch.float64, grad_fn=<MseLossBackward0>)


In [4]:
params = torch.load("0.00189921_drugsOnly_leakyReLu_960_1100_model_best.pth")
# Get the dimensions of the first layer and the weights
latent_size, input_size = params['state_dict']['net.0.weight'].shape
weights_encode = torch.nn.Parameter(params['state_dict']['net.0.weight'])

# Initialize an encoding layer, apply weights
to_latent_nn = nn.Linear(input_size, latent_size, bias=False)
to_latent_nn.weight = weights_encode
to_latent_nn.double()
to_latent_nn.eval()

pEF_vector_latent = to_latent_nn(pEF_vector_in)

In [5]:
pd.DataFrame(pEF_vector_latent.detach().numpy(), columns = ['pEF_vector']).to_csv('HFpEF_latent_vector.csv', index = False)